# RespInPeace: Toolkit for precessing breathing belt (RIP) data

In [ ]:
import datetime
import matplotlib.pyplot as plt
import peakutils

# Silence FutureWarnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

%matplotlib inline

## Data loading

Load data from a WAV file and plot the respiratory signal:

In [ ]:
from rip import Resp

resp = Resp.from_wav('resp.wav')

print('''Sampling frequency: {}
Number of samples: {}
Duration: {}'''.format(resp.samp_freq, len(resp.samples), datetime.timedelta(seconds=resp.dur)))

In addition, reading the respiratory data from a CSV file listing is also supported using the `from_csv` constructor. See the documentation for how to use it.

While **RespInPeace** provides ways to automatically identify inhalations and exhalations in the respiratory signal, it is also possible to use one's own respiratory segmentation by specifying the `cycles` argument. See the documentation for details.

## Iteration, indexing and slicing

Time stamp and respiratory values are stored in `t` and `resp` attributes, respectively:

In [ ]:
resp.t[:5], resp.samples[:5]

In [ ]:
plt.plot(resp.t / 60, resp.samples)

RIP objects can be indexed and sliced both by sample and time indices:

In [ ]:
resp[10:20:2]

In [ ]:
resp.idt[0.05:0.1:0.01]

They can also be used as iterators:

In [ ]:
for i, samp in enumerate(resp):
    if i > 10:
        break
    print(i, samp)

# Detrending and drift removal

**RespInPeace** implements several detrending methods:
- Subracting the mean with `detrend(type='constant')`
- Removal of linear trend with `detrend(type='linear')`
- Removal with low-frequency baseline oscilation using a square window (`remove_basline_square`), a Savitzky-Golay filter (`remove_baseline_savgol` or Asymmetric Least Squares Smoothing (`remove_baseline_als`).

Below we demonstrate the last of these methods:

In [ ]:
resp.remove_baseline_als()
plt.plot(resp.samples)

In addition to these generic drift removal methods, RestInPeace also implements baseline detection based on dynamic estimation of REL (see below).

## Signal segmentation

### Detection of inhalations and exhalations

Inhalations and exhalations can be identified using the `find_cycles` method:

In [ ]:
resp.find_cycles(include_holds=False)

In [ ]:
type(resp.segments)

In [ ]:
resp.segments[:10]

It is also possible to access inhalations and exhalations, separately:

In [ ]:
resp.inhalations[:5]

In [ ]:
resp.exhalations[:5]

Finally, we can extract peak and trough times, stored in numpy arrays:

In [ ]:
resp.peaks[:5]

In [ ]:
resp.troughs[:5]

Here is a plot showing the first couple of cycles.

In [ ]:
plt.plot(resp.t, resp.samples, color='lightgrey')
plt.plot(resp.peaks, resp.idt[resp.peaks],
         linestyle='none', marker='o')
plt.plot(resp.troughs, resp.idt[resp.troughs],
         linestyle='none', marker='o')
plt.xlim(0,  1e4 /  resp.samp_freq)

Given this representations, calculating durational features is extremely easy. For instance, below  we calculate and plot the distribution of inhalation durations:

In [ ]:
inh_durs = [i.duration() for i in resp.inhalations]

We can also visualise them:

In [ ]:
_ = plt.hist(inh_durs, bins=20)

### Hold detection

Hold detection follows an adapted version of the method implemented in [Breathmetrics](https://github.com/zelanolab/breathmetrics): For each respiratory segment (inhalation and exhalation) we construct a histogram of respiratory values. Since respiratory holds result is speakers staying at the same lung volume for a while, they are identified as peaks in the histogram.

The specifics of hold detection can be controlled arguments passed to the `find_holds` method:
* `bins` (default: 100) - the number of bins used to construct the histogram.
* `peak_prominence` (default 0.05) - the minimal peak prominence (as returned by [`scipy.signal.peak_prominences`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.peak_prominences.html)) corresponding to a respiratory hold.
* `min_hold_gap` (default: 0.15) - neighbouring holds separated by a gap shorter than this value will be merged.
* `min_hold_dur` (default: 0.25 s) - holds shorter than this value will be omitted. Note that this criterion is applied after merging of neighbouring holds using the value of `min_hold_gap` above.

In [ ]:
resp.find_holds()
resp.holds[:10]

Below, we plot the first four holds for illustrative purposes:

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(15, 10))

mar = 5
for i, ax in enumerate(axs.flatten()):
    hold = resp.holds[i]
    ax.plot(resp.t, resp.samples, color='grey')
    ax.axvspan(hold.start_time, hold.end_time, alpha=0.3)
    ax.set_xlim(hold.end_time - mar, hold.end_time + mar)

## Range estimation

Respiratory range is estimated in terms of the bottom and top percentiles of peak and trough respiratory values. By default, the 5th and 95 percentiles are used. In other words, the bottom- and top-most 5 per cent of peaks and troughs are discarded.

In [ ]:
resp.estimate_range()

plt.plot(resp.samples, color='lightgrey')
plt.axhline(resp.range_bot, linestyle='dashed')
plt.axhline(resp.range_top, linestyle='dashed')

## REL estimation

Resting expiratory level(REL) is the state of equilibrium when the expiratory elasticity forces equal the the inhalatory elasticity forces. It also the point within the total lung capacity where speakers are most likely to inhale as speaking at lung volumes below REL is generally felt to be quite uncomfortable.

RespInPeace offers two ways of estimating REL. By default, it will use a *static* REL value calculated as the median of all respiratory troughs. REL is quite sensitive to posture shifts and if the subject did not stand perfectly still, might fluctuate in the course of a recording. In order to neutralize the effect this drift, REL needs to be estimated in a more dynamic way. RestInPeace estimates REL by calculating the median level of all troughs in a window of specified size (by default, `win_len=11`, i.e. REL is calcualte as the median level of five preceding and five following troughs).

In [ ]:
resp.estimate_rel(dynamic=True, win_len=11)

plt.plot(resp.t, resp.samples, color='lightgrey')
plt.axhline(y=0)

In [ ]:
plt.plot(resp.t, resp.samples, color='lightgrey')
plt.axhline(y=0)
plt.xlim(30, 200)

## Feature extraction

RespInPeace can be used to extract a number of respiratory features from a selected interval or sample, such as amplitude (`extract_amplitude`) and slope (`extract_slope`). These features can be calculated  either relative to the estimated respiratory range (`norm=True`, the default) or raw (`norm=False`). In addition, `extract_features` can be used to extract all the implemeneted features.

Below, we demonstrate extraction of features from the first inhalation in the file:

In [ ]:
inh = resp.inhalations[0]
resp.extract_features(inh.start_time, inh.end_time, norm=True)

More complex examples of feature extraction can be found in [scripts/](../scripts).

## Writing to files

### Respiratory data

The respiratory signal can be save to a file with the `save_resp` method. By default, the data is saved to a WAV file.

In [ ]:
resp.save_resp('resp_saved.wav')

Alternatively, the data can be saved to a CSV file by passing a `filetype="table"` argument to `save_resp`. See the documentation for more details.

### Annotations

The annotations (inhalation, exhalation and hold boundaries) can be saved to a file with the `save_annotations` method. By default, they are saved to a [PRAAT](http://www.fon.hum.uva.nl/praat/) TextGrid files (the so-called short format). Other supported formats are `"eaf"` (for [ELAN](https://tla.mpi.nl/tools/tla-tools/elan/) EAF XML-based format) and `"table"` for a CSV (comma-separated) file. The tiers to save can be specified using the `tiers` argument to `save_annotations`.

In [ ]:
resp.save_annotations('resp.TextGrid', tiers=['cycles', 'holds'])

If `merge_holds` is True, the holds are merged with the respiratory segmentation. In that case, the holds tier is not included in the output.